# Longtype
* 차량 쌍(TV, LV) 기준으로 정리하기
* 왜냐하면, 각 쌍별로 SSM이 계산되기 때문임

# Import

In [1]:
import pandas as pd
import numpy as np

from tqdm import tqdm
import warnings
import os

from scipy import stats # Z-score를 이용한 이상값 제거

import math # arctangent; math.atan 사용 목적
import statistics

import pyarrow.parquet as pq
#df = pq.read_pandas('data.parquet').to_pandas()

# Load Dataset

In [2]:
working_dir = 'D:/OneDrive/Projects/2023_SSM_Feasibility/Dataset'

In [3]:
file_list = ['LV_trajectories-0750am-0805am.parquet']

In [4]:
folder_name = '02_processed'

for file, i in zip(file_list, [1]):
    file_path = os.path.join(working_dir, folder_name, file)
    globals()[f'df_{i}'] = pq.read_pandas(file_path).to_pandas()

In [5]:
df_1.head(3)

,veh_id,frm,local_y,local_x,V_len,V_wid,velocity,acc,lane,Preceeding,...,LVR_delta_velocity_y,LVR_acc_x,LVR_acc_y,LVR_Lane_record,LVR_Lane_record_split,LVR_Lane_change_direction,LV0_D,LVL_D,LVR_D,LC_CF
index,,,,,,,,,,,,,,,,,,,,,
0,2,13,17.375000,10.78125,4.417969,1.493164,43.90625,0.0,U02,0,...,-0.000847,0.004704,-0.002352,U04,U04_U04,Straight,NaN,NaN,10.132812,LC
1,2,14,17.375000,12.00000,4.417969,1.493164,43.90625,0.0,U02,0,...,0.000000,-0.002352,0.000000,U04,U04_U04,Straight,NaN,NaN,10.125000,LC
2,2,15,17.390625,13.21875,4.417969,1.493164,43.90625,0.0,U02,0,...,0.001694,-0.000000,0.004704,U04,U04_U04,Straight,NaN,NaN,10.132812,LC


# Longtype
* pairs 리스트의 튜플(TV, LV)에 따라서 각 차량의 위치, 속도, 가속도 등을 df로부터 프레임별로 뽑아오기

In [6]:
for df, i in tqdm(zip([df_1], [1])):

    df_LV0 = df[['veh_id', 'LV0_ID', 'frm']].copy()
    df_LV0 = df_LV0.rename({'LV0_ID' : 'LV_ID'}, axis =1)
    df_LV0['LV_type'] = 'LV0'
    
    df_LVL = df[['veh_id', 'LVL_ID', 'frm']].copy()
    df_LVL = df_LVL.rename({'LVL_ID' : 'LV_ID'}, axis = 1)
    df_LVL['LV_type'] = 'LVL'

    df_LVR = df[['veh_id', 'LVR_ID', 'frm']].copy()
    df_LVR = df_LVR.rename({'LVR_ID' : 'LV_ID'}, axis = 1)
    df_LVR['LV_type'] = 'LVR'
    
    total_long = pd.concat([df_LV0, df_LVL, df_LVR])
    
    # LV_ID가 0인 것은 제거한다
    total_long = total_long[(total_long['LV_ID'] != 0) & (pd.isna(total_long['LV_ID']) == False)]
    total_long['LV_ID'] = total_long['LV_ID'].astype('int')
    total_long.reset_index(inplace = True, drop = True)
    
    globals()[f'long_{i}'] = total_long

1it [00:00,  3.74it/s]


In [7]:
veh_pair = long_1[['veh_id', 'LV_ID']].drop_duplicates()

In [8]:
def make_int(x):
    if pd.isna(x) == False:
        return int(x)

    else:
        return None

In [9]:
def RL0(lane, LV_lane):
    if pd.isna(lane) == False and pd.isna(LV_lane) == False:
        lane_num = int(lane[1:])
        LV_lane_num = int(LV_lane[1:])

        if lane_num == LV_lane_num:
            type_LV = 'LV0'
            
        elif lane_num < LV_lane_num:
            type_LV = 'LVR'
            
        elif lane_num > LV_lane_num:
            type_LV = 'LVL'

        else:
            type_LV = None

    else:
        type_LV = None

    return type_LV

In [10]:
warnings.filterwarnings('ignore')

target_df_cols = ['local_x', 'local_y', 'V_len', 'V_wid', 'velocity', 'acc', 'lane', 'Time',
                  'local_x_before', 'local_y_before', 'delta_local_x', 'delta_local_y',
                  'velocity_x', 'velocity_y', 'velocity_x_before', 'velocity_y_before', 'delta_velocity_x', 'delta_velocity_y',
                  'acc_x', 'acc_y', 'lane_past', 'Lane_record', 'Lane_record_split', 'Lane_00', 'Lane_99',
                  'Lane_change', 'Lane_leave', 'Lane_change_direction', 'LC_CF']

column_order = ['pair', 'veh_id', 'LV_ID', 'frm', 'LV_type', 'local_x', 'local_y', 'V_len', 'V_wid', 'velocity',
       'acc', 'lane', 'Time', 'local_x_before', 'local_y_before',
       'delta_local_x', 'delta_local_y', 'velocity_x', 'velocity_y',
       'velocity_x_before', 'velocity_y_before', 'delta_velocity_x',
       'delta_velocity_y', 'acc_x', 'acc_y', 'lane_past', 'Lane_record',
       'Lane_record_split', 'Lane_00', 'Lane_99', 'Lane_change', 'Lane_leave',
       'Lane_change_direction', 'LC_CF', 'LV_local_x',
       'LV_local_y', 'LV_len', 'LV_wid', 'LV_velocity', 'LV_acc', 'LV_lane',
       'LV_Time', 'LV_local_x_before', 'LV_local_y_before', 'LV_delta_local_x',
       'LV_delta_local_y', 'LV_velocity_x', 'LV_velocity_y',
       'LV_velocity_x_before', 'LV_velocity_y_before', 'LV_delta_velocity_x',
       'LV_delta_velocity_y', 'LV_acc_x', 'LV_acc_y', 'LV_lane_past',
       'LV_Lane_record', 'LV_Lane_record_split', 'LV_Lane_00', 'LV_Lane_99',
       'LV_Lane_change', 'LV_Lane_leave', 'LV_Lane_change_direction',
       'LV_LC_CF', 'D_x', 'D_y', 'D', 'D_gap']


##### 엇갈림구간 임시저장폴더 : 02_temp_weave ########
# 차량별로 저장 후 concat은 일괄수행해야 한다.

#### 차량 페어링하기기
for long, i in zip([long_1], [1]):

    # 레퍼런스 데이터프레임
    reference_df = globals()[f'df_{i}'][['veh_id', 'frm'] + target_df_cols].copy() # Merge의 reference가 될 데이터프레임
    veh_list = list(reference_df['veh_id'].unique())

    veh_tot = pd.DataFrame()
    
    for i in tqdm(range(len(veh_pair))): # 각 차량에 대하여
        row = veh_pair.iloc[i] # 차량 쌍
        veh_id = row['veh_id']
        LV_ID = row['LV_ID']
        #LV_type = row['LV_type']

        veh_TV = reference_df[reference_df['veh_id'] == veh_id].copy()
        veh_LV = reference_df[reference_df['veh_id'] == LV_ID].copy()

        # LV 데이터프레임 컬럼이름을 적절하게 변경
        for col in target_df_cols:
            veh_LV = veh_LV.rename({col : 'LV_'+col}, axis = 1)

        veh_LV.rename({'LV_V_len' : 'LV_len', 'LV_V_wid' : 'LV_wid'}, axis = 1, inplace = True)

        veh_TV['LV_ID'] = LV_ID
        #veh_TV['LV_type'] = LV_type
        veh_tot = pd.merge(veh_TV, veh_LV, how = 'left', left_on = ['LV_ID', 'frm'], right_on = ['veh_id', 'frm']).rename({'veh_id_x' : 'veh_id'}, axis = 1).drop(['veh_id_y'], axis = 1)
        pair_id = str(veh_id) + '_' + str(LV_ID)

        veh_tot['pair'] = pair_id

        # LV_type 매기기
        veh_tot['LV_type'] = veh_tot.apply(lambda row: RL0(row['lane'], row['LV_lane']), axis = 1)

        # 거리관련 변수
        veh_tot['D_x'] = veh_tot['LV_local_x'] - veh_tot['local_x']
        veh_tot['D_y'] = veh_tot['LV_local_y'] - veh_tot['local_y']
        veh_tot['D'] = (veh_tot['D_x']**2 + veh_tot['D_y']**2) ** (1/2)
        veh_tot.reset_index(inplace = True, drop = True)
        
        veh_tot['D_lag'] = veh_tot['D'].shift(1) # 직전 row의 값임
        veh_tot['veh_id'] = veh_tot['veh_id'].astype('int')
        veh_tot['LV_ID'] = veh_tot['LV_ID'].apply(make_int)
        veh_tot['frm'] = veh_tot['frm'].astype('int')
        
        #veh_tot['veh_id_lag'] = veh_tot['veh_id'].shift(1) # 직전 row 값
        #veh_tot['LV_ID_lag'] = veh_tot['LV_ID'].shift(1) # 직전 row 값
        
        veh_tot['D_gap'] = veh_tot['D'] - veh_tot['D_lag'] # 거리 변화. (-)이면 줄어들고 있다는 뜻이다
        
        # 만약 veh_id, LV_id 의 lag값이 현재 row와 다르다면, 새로운 값이 시작된 것이므로 D_gap을 NaN으로 만들어줘야 한다.
        #veh_tot.loc[(veh_tot['veh_id'] != veh_tot['veh_id_lag']) | (veh_tot['LV_ID'] != veh_tot['LV_ID_lag']), 'D_gap'] = None
        
        #print(f'{i} : {len(long_final)}')
        veh_tot.drop(['D_lag'], axis = 1, inplace = True)
        veh_tot = veh_tot[column_order]
        veh_tot = veh_tot.sort_values(by = 'pair').reset_index(drop = True)        

        temp_save_name = f'{pair_id}_weave.parquet'
        temp_save_path = os.path.join(working_dir, '02_weave_temp', temp_save_name)
        veh_tot.to_parquet(temp_save_path, engine = 'fastparquet', compression = 'gzip') 

100%|██████████████████████████████████████████████████████████████████████████| 44278/44278 [1:01:40<00:00, 11.97it/s]


* ** Merge Small Parquet to Huge Parquet** : https://stackoverflow.com/questions/75468395/merge-small-parquet-files-into-a-single-large-parquet-file

In [12]:
#### 굳이 수행하지 않아도 되는 병합코드. 이건 나중에 한다.

# save_folder = '02_processed_long'
# save_files = ['LV_long_trajectories-0750am-0805am.parquet']

# file_list = os.listdir(os.path.join(working_dir, '02_weave_temp'))

# save_path = os.path.join(working_dir, save_folder, save_files[0])

# schema_path = os.path.join(working_dir, '02_weave_temp', file_list[0])
# schema = pq.ParquetFile(schema_path).schema_arrow

# save_path = os.path.join(working_dir, save_folder, save_files[0])

# with pq.ParquetWriter(save_path, schema = schema) as writer:
#     for file in tqdm(file_list):
#         file_path = os.path.join(working_dir, '02_weave_temp', file)
#         writer.write_table(pq.read_table(file_path, schema = schema))